Оценка качества тематических эмбеддингов, основанная на задаче определения перефразирования. Будет использован общедоступный корпус парафраз (проект http://paraphraser.ru/). 


## download the Paraphraser dataset

Сначала нужно скачать и предобработать этот корпус. 

Предобработка: парсинг XML, токенизация, лемматизация, подсчёт частот слов внутри одного документа, создание `TopicNet.Dataset` (который будет хранить информацию о частотах слов внутри всего корпуса).


In [ ]:
! wget http://paraphraser.ru/download/get?file_id=1
! unzip get?file_id=1

In [ ]:
from lxml import etree

with open("paraphrases.xml", "rb") as f:
    corpus = f.read()

PT = etree.fromstring(corpus)


In [ ]:
classes = {}
texts = {}

for i, paraphrase in enumerate(PT.getchildren()[1].getchildren()):
    data = {
        child.get('name'): child.text 
        for child in 
        paraphrase.getchildren()
    }
    key = data['id_1'], data['id_2']
    classes[key] = data['class']
    texts[key[0]] = data['text_1']
    texts[key[1]] = data['text_2']


In [ ]:
import pandas as pd
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()



In [ ]:
from collections import Counter

def vowpalize_sequence(sequence):
    word_2_frequency = Counter(sequence)
    del word_2_frequency['']
    vw_string = ''

    for word in word_2_frequency:
        vw_string += word + ":" + str(word_2_frequency[word]) + ' '
    return vw_string

In [ ]:
import re

def _find_next_token_func(line, start_ind, regexp):
    m = regexp.search(line, start_ind)
    if m:
        start_ind, length = m.start(), len(m.group())
    else:
        start_ind, length = start_ind, 0
    return start_ind, length

def find_indexes(string, regexp):
    """
    Find indexes of all tokens in string

    Parameters
    ----------
    string : str
        String, supposed to be a sentence or something analogous

    Return
    ------
    index_list : list of int
        List of indexes. Even indexes are word start positions, uneven indexes are word lengths.
    """
    index_list = []
    start_ind, length = 0, 0
    while True:
        start_ind, length = _find_next_token_func(string, start_ind + length, regexp)
        if length == 0:
            break
        index_list.append((start_ind, length))
    return index_list

In [ ]:
BASE_RU_TOKEN_REGEX = re.compile(
    '''(?:-|[^a-zа-яё\s"'""«»„“-]+|[0-9a-zа-яё_]+(-?[0-9a-zа-яё_]+)*)''',
    re.IGNORECASE | re.UNICODE)
import string


def tokenize(the_string, regexp=BASE_RU_TOKEN_REGEX):
    index_list = find_indexes(the_string, regexp)

    tokenized_string = [
        the_string[ind_start:ind_start + length]
        for ind_start, length in index_list
    ]
    return [part.replace(":", "%3A") for part in tokenized_string if part not in string.punctuation]



In [ ]:
lemmatized_paraphraser_dataset = pd.DataFrame(index=texts.keys(), columns=['raw_text', 'vw_text'])



for idx, text in texts.items():
    sequence = [morph.parse(w)[0][2] for w in tokenize(text)]
    lemmatized = '@lemmatized ' + vowpalize_sequence(sequence)
    vw_string = ' |'.join([idx, lemmatized])
    
    lemmatized_paraphraser_dataset.loc[idx, 'raw_text'] = text
    lemmatized_paraphraser_dataset.loc[idx, 'vw_text'] = vw_string


lemmatized_paraphraser_dataset.index.rename("id", inplace=True)

Сохраним результаты (датасет и метки классов).


In [ ]:
import ujson


with open("classes_paraphraser.json", "w") as f:
    ujson.dump(classes, f)

In [ ]:
from topicnet.cooking_machine import Dataset


dataset = Dataset.from_dataframe(lemmatized_paraphraser_dataset, "./paraphraser_dataset")
dataset._data.to_csv("paraphraser_dataset.csv")

## load the Paraphraser dataset

Загрузим датасет и метки классов, вычислим некоторые статистики частот слов, скачаем модель из библиотеки ОБВПТМ и изучим её поведение на данной задаче.

In [1]:
from ast import literal_eval


import ujson


with open("classes_paraphraser.json", "r") as f:
    classes_raw = ujson.load(f)
    
classes = {
    literal_eval(k): literal_eval(v)
    for k, v in classes_raw.items()
}

In [2]:
from collections import Counter
from topicnet.cooking_machine.dataset import dataset2counter

from topicnet.cooking_machine import Dataset


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr, spearmanr

dataset_paraphraser = Dataset("paraphraser_dataset.csv",  internals_folder_path="./paraphraser_dataset")


In [5]:
paraphraser_counter = dataset2counter(dataset_paraphraser)

In [6]:
from topicnet.embeddings.keyed_vectors import (
    get_doc_vec_phi, get_doc_vec_keyedvectors, topic_model_to_keyed_vectors, calc_dataset_statistics
)

/data/mytopicnet/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [7]:

dict_parap = calc_dataset_statistics(dataset_paraphraser)

In [8]:
from topicnet.cooking_machine.models import TopicModel


any_model = load_model("ARTM_150_Base")

Метки классов в корпусе принимают одно из трёх возможных значений: 
* -1: разные по смыслу новости  ("80% жителей России поддерживают антитабачный закон" и "Госдума приняла антитабачный законопроект во втором чтении")
* 0: похожие по смыслу новости ("ЦИК хочет отказаться от электронной системы подсчета голосов" и "ЦИК может отказаться от электронной системы подсчета голосов")
* 1: одинаковые по смыслу новости ("СК выяснит, был ли подкуп свидетеля по делу Ю.Буданова" и "СК проверит информацию о подкупе свидетеля по делу об убийстве Буданова")

Для того, чтобы оценить качество эмбеддингов, будем измерять корреляцию Спирмана между эталонными метками и метками, предсказанными моделью (как косинусная близость между эмбеддингами заголовков).


In [9]:

def measure_document_task_avg(model, phi, classes, dict_df, counter, avg_scheme="unit"):
    predicted = []
    true_labels = []

    for pair, value in classes.items():
        v1 = get_doc_vec_phi(phi, counter[pair[0]], dict_df, avg_scheme).values.reshape(1, -1)
        v2 = get_doc_vec_phi(phi, counter[pair[1]], dict_df, avg_scheme).values.reshape(1, -1)
        predicted_val = cosine_similarity(v1, v2)[0][0]
        value = int(value)
        true_labels.append(value)
        # bins[value].append(predicted_val)
        predicted.append(predicted_val)

    return spearmanr(predicted, true_labels)[0]

        
def measure_document_task_theta(model, classes, dataset):
    theta = model.get_theta(dataset=dataset)
    predicted = []
    true_labels = []

    sp = model.specific_topics

    for pair, value in classes.items():
        v1 = (theta.loc[sp, pair[0]].values.reshape(1, -1))
        v2 = (theta.loc[sp, pair[1]].values.reshape(1, -1))
        predicted_val = cosine_similarity(v1, v2)[0][0]
        value = int(value)
        true_labels.append(value)
        # bins[value].append(predicted_val)
        predicted.append(predicted_val)

    return spearmanr(predicted, true_labels)[0]


In [10]:
measure_document_task_avg(any_model, any_model.get_phi(), classes, dict_parap, paraphraser_counter, "unit")

0.3525540032601885

In [11]:
measure_document_task_avg(any_model, any_model.get_phi(), classes, dict_parap, paraphraser_counter, "tf-idf")

0.37121453978036173

In [12]:
measure_document_task_theta(any_model, classes, dataset_paraphraser)

0.3045784415119519

Видим, что документные эмбеддинги, полученные из столбцов матрицы Тета (то есть вычисленные при помощи ЕМ-алгоритма), уступают по качеству документным эмбеддингам, построенным при помощи усреднения векторов слов. Также отметим, что усреднение при помощи tf-idf весов увеличивает качество по сравнению с "простым" усреднением.


## Traditional embeddings

ОБВПТМ может быть полезен и для работы с традиционными (не-тематическими) эмбеддингами, поскольку предоставляет возможность вычислять эмбеддинг документа различными способами (что не позволяет сделать GenSim).


In [13]:
import navec

path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'

vec = navec.Navec.load(path)


In [14]:


def measure_document_task_kv(vec, classes, dict_df, counter, avg_scheme="unit"):
    predicted = []
    true_labels = []

    for pair, value in classes.items():
        v1 = get_doc_vec_keyedvectors(vec, counter[pair[0]], dict_df, avg_scheme).values.reshape(1, -1)
        v2 = get_doc_vec_keyedvectors(vec, counter[pair[1]], dict_df, avg_scheme).values.reshape(1, -1)
        predicted_val = cosine_similarity(v1, v2)[0][0]

        value = int(value)
        true_labels.append(value)
        predicted.append(predicted_val)
    return spearmanr(predicted, true_labels)[0]




In [15]:
qual_tfidf = measure_document_task_kv(vec, classes, dict_parap, paraphraser_counter, "tf-idf")

qual_base = measure_document_task_kv(vec, classes, dict_parap, paraphraser_counter, "unit")

print(qual_base, qual_tfidf)

0.5617694371502122 0.5994033884599912


Остаётся справедливым наблюдение о том, что усреднение при помощи tf-idf весов увеличивает качество по сравнению с "простым" усреднением.


ОБВПТМ позволяет преобразовать тематическую модель в объект `Gensim.KeyedVectors` и работать с этим объектом, как с традиционным эмбеддингом:

In [16]:
vec2 = topic_model_to_keyed_vectors(any_model, "@lemmatized")

In [17]:
qual_tfidf = measure_document_task_kv(vec2, classes, dict_parap, paraphraser_counter, "tf-idf")

qual_base = measure_document_task_kv(vec2, classes, dict_parap, paraphraser_counter, "unit")

print(qual_base, qual_tfidf)

0.30905862857638616 0.3148957860011381


(разница численных значений связана с неодинаковой обработкой фоновых тем)